In [8]:
from konlpy.tag import Okt
from gensim import corpora
import re

# 1. 파일 읽기
with open("KakaoTalk_20250515_0053_22_930_유정유정.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 2. 메시지 추출 (날짜/이름/시간 제거)
# 형식 예시: [김소연] [오후 3:36] 정산하기를 요청했어요.
messages = re.findall(r'\[.+?\] \[.+?\] (.+)', raw_text)

print(messages)

# 3. 전처리 함수 정의  
okt = Okt()

def preprocess(text):
    # 특수문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    # 명사만 추출
    nouns = okt.nouns(text)
    # 길이 1 이하 제거
    return [word for word in nouns if len(word) > 1]

# 4. 전체 문서 리스트 생성
tokenized_docs = [preprocess(msg) for msg in messages if msg.strip()]

# 5. 딕셔너리 및 코퍼스 생성
dictionary = corpora.Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# 전처리 결과 확인
print(tokenized_docs)

['유정앙 아직 출발 안했으면 한 20분만 미룰수 이쓰까..ㅠㅠ', '하던게 안끝났어.. 쏘리..', '헉 아냐아냐!!?', '여유있게 와 ㅎㅎ', '30분쯤 보쟝!!', '앗 고마어.. 그때 보쟝!!', '나 도착했어!!', '어디서 볼까??', '헉 !!! 나는 버스가 늦어져서 곧 탈 거 같아...ㅎㅎ', '그 스타벅스 갈까 하는데 어때??', '[네이버 지도]', '구랭 그쪽으로 가고있을게!!', '웅웅 고마어🥰🥰', '언능 갈게!!!', '나 내렸오 ㅎㅎ', '스벅 앞이양??', '웅웅', '근데 여기 자리가 업따..', '미친', '건너편에 투썸 있는데 가볼까???', '아 ㅇㅋ 보인다', '내가 지금 투썸 근천데', '아하', '건너 올랭?????', '웅웅', '알써😆😆', '파일: KakaoTalk_20250511_1625_29_658_유정유정.txt', 'TF/TF-IDF 기반', '사진', '3,500원을 보냈어요.', '3,500원을 받았어요.', 'https://www.genspark.ai/', 'gimsoyeon092@gmail.com', '언니 진짜 너무 고샹해써... 조심히 잘 들어가구 화욜에 보자😂😂❤', '너두 오늘 너무너무 고생해써..!! 푹 쉬고 화요일에 보쟝~', '언니 근데 우리 아직 자주 쓰는 단어랑 종결어미 분석하는 건 구현이 안 되어 있자나 이건 추후에 클러스터링 이용해서 추가할거라고 말하는 게 좋을 거 같은데 오때??', '웅 좋아좋아~ 추후 진행할 부분에서 LDA 말할때 하면 되겠다!', '오케이!! 고마웡🥰', '언니 !! 나 대본 다썼어 ㅎㅎ', '읽어봤어!! 고생해써~ 이대로 하면 될것 같은데!?', '오 진짜??? 조아조아 고마어 ㅎㅎㅎㅎ', '내일 이대로 할겡 ㅎㅎ', '언니 혹시 코드 분석 얼마나 했어?? 내가 아직 학교라서 집 가서 할 수 있을 거 같아….', '곧 집으로 출발해..ㅠㅠ', '아 헐 나 오늘 계속 자버렸네.. 지금부터 하고 있을게ㅠㅠ', '유정앙 오늘꺼는 내가 해서 제출할겡~ 금방 할수

In [15]:
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# 1. 파일 읽기
with open("KakaoTalk_20250515_0053_22_930_유정유정.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 2. 메시지 추출
messages = re.findall(r'\[.+?\] \[.+?\] (.+)', raw_text)

# 3. NER 사전학습 모델 로딩
model_name = "klue/bert-base"  # ✅ 반드시 NER 학습된 모델 사용
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# 4. 파이프라인 생성
ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# 5. 메시지별 명사 추출
for message in messages:
    ner_results = ner_pipe(message)
    for entity in ner_results:
        word = entity['word']
        label = entity['entity_group']
        if label in ['PER', 'ORG', 'LOC']:  # 인물, 조직, 장소
            print(f"명사 후보: {word} (라벨: {label})")

ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
from gensim import models
import pyLDAvis.gensim_models
import pyLDAvis
import matplotlib.pyplot as plt

# 6. LDA 모델 학습
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,         # 추출할 주제 수
    random_state=42,
    passes=10,            # 반복 횟수
    per_word_topics=True
)

# 7. 주제 출력
for idx, topic in lda_model.print_topics(num_words=5):
    print(f"🧩 주제 {idx + 1}: {topic}")

# 8. pyLDAvis로 시각화
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, 'lda_kakaotalk_result.html')
print("📁 LDA 결과가 lda_kakaotalk_result.html 파일로 저장되었습니다.")

🧩 주제 1: 0.073*"보쟝" + 0.050*"고마" + 0.050*"진짜" + 0.027*"언니" + 0.027*"분석"
🧩 주제 2: 0.032*"고생" + 0.032*"할겡" + 0.032*"언니" + 0.032*"대본" + 0.032*"제출"
🧩 주제 3: 0.053*"고마" + 0.053*"할겡" + 0.053*"부분" + 0.053*"진행" + 0.053*"오케이"
🧩 주제 4: 0.061*"진짜" + 0.061*"유정" + 0.061*"고생" + 0.061*"웅웅" + 0.042*"실행"
🧩 주제 5: 0.045*"투썸" + 0.045*"웅웅" + 0.045*"지금" + 0.045*"출발" + 0.045*"내일"
📁 LDA 결과가 lda_kakaotalk_result.html 파일로 저장되었습니다.
